In [ ]:
# load packages
# make sure to install the pacakge "tqdm" for the progress bar when training.
import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.nn.functional as F
from torch.autograd import Variable
from torchvision.utils import save_image
from tqdm import tqdm
import numpy as np
import matplotlib.pyplot as plt
import imageio
import matplotlib.image as mpimg
from scipy import ndimage

path_prefix = ""

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
path_prefix = "/content/gdrive/MyDrive/CMPT 726-419 Spring 2021 A2"

import sys
sys.path.insert(1, path_prefix)

In [ ]:
class Autoencoder(nn.Module):

    def __init__(self,dim_latent_representation=2):

        super(Autoencoder,self).__init__()

        class Encoder(nn.Module):
            def __init__(self, output_size=2):
                super(Encoder, self).__init__()
                # needs your implementation
                self.layer1 = nn.Linear(784, 1024)
                self.layer2 = nn.ReLU()
                self.layer3 = nn.Linear(1024, 10)
                pass

            def forward(self, x):
                # needs your implementation
                output = x.view(x.size(0), 784)
                output = self.layer1(output)
                output = self.layer2(output)
                output = self.layer3(output)
                return output
                pass

        class Decoder(nn.Module):
            def __init__(self, input_size=2):
                super(Decoder, self).__init__()
                # needs your implementation
                self.layer1 = nn.Linear(10, 1024)
                self.layer2 = nn.ReLU()
                self.layer3 = nn.Linear(1024, 784)
                pass

            def forward(self, z):
                # needs your implementation
                output = self.layer1(z)
                output = self.layer2(output)
                output = self.layer3(output)
                output = torch.sigmoid(output)
                output = output.view(-1, 1, 28, 28)
                return output
                pass

        self.encoder = Encoder(output_size=dim_latent_representation)
        self.decoder = Decoder(input_size=dim_latent_representation)

    def forward(self,x):
        x = self.encoder(x)
        x = self.decoder(x)
        return x

In [ ]:
from autoencoder_starter import Autoencoder_Trainer

LEARNING_RATE = 1e-3
EPOCH_NUMBER= 10 # the number of epochs and learning rate can be tuned.

autoencoder = Autoencoder(dim_latent_representation=2)
trainer = Autoencoder_Trainer(autoencoder_model=autoencoder,learning_rate=LEARNING_RATE,path_prefix=path_prefix)

try:
    for epoch in range(1, EPOCH_NUMBER + 1):
        trainer.train(epoch)
        trainer.validate(epoch)
except (KeyboardInterrupt, SystemExit):
        print("Manual Interruption")

In [ ]:
images = trainer.get_val_set()
from autoencoder_starter import display_images_in_a_row
for t in np.linspace(0,10,11):
  x = np.add((t/10)*images[0], (1-(t/10))*images[1])
  if t == 0:
    output = x
  else:
    output = torch.cat((output,x))
display_images_in_a_row(output.cpu())